<a href="https://colab.research.google.com/github/s1c5000/kaggle/blob/main/Titanic/4_2_Introduction_to_Ensembling_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

캐글 필사4
* 원본 : https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

In [ ]:
# kaggle json
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 56.3MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.26MB/s]


# Introduction
이 노트북은 기본 학습모델을 병합 하는 방법, 특히 stacking에 대한 매우 기본적이고 간단한 입문서 입니다. 스태킹은 간단히 말해서 처음(기본) 모델이 예측한 Prediction을 second모델의 학습에 사용하는 것 입니다. 많은 우승자가 stacking을 사용하여 우승 하였지만, stacking에 대한 노트북은 적은 것 같아 해당 노트북이 그 공백을 채우면 좋겠습니다     
이 스크립트는 0.808의 LB점수를 제공하며 상위9%에 도달하고 4분 미만으로 실행됩니다

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(font_scale=2)

import re
import sklearn
import xgboost as xgb

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

# Feature Exploration, Engineering and Cleaning
대부분의 커널이 구조화되는 방식과 매우 유사하게 진행할 것입니다. 즉 먼저 데이터를 탐색하고 feature엔지니어링 하고 모든 categorical feature을 수치형으로 인고딩 합니다.


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

PassengerId = test['PassengerId']
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


우리의 임무는 어떻게는 범주형 변수에서 정보를추출 하는 것 입니다.

# Feature Engineering
Titanic_top4%_with_ensemble_modeling.ipynb의 feature Engineering을 이용하겠습니다

In [ ]:
######## quartile을 이용해 outlier값 제거
from collections import Counter
def detect_outliers(df, n, features):
  # 리스트로반환
  outlier_indices = []

  for col in features:
    # 1st quartile (25%)
    Q1 = np.percentile(df[col], 25)
    # 3rd quartile 75%
    Q3 = np.percentile(df[col], 75)
    # Interquartile range (IQR)
    IQR = Q3 - Q1

    #outlier step
    outlier_step = 1.5 * IQR

    # 특정 col에 대해 outlier를 찾는다
    outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q1 + outlier_step)].index
    outlier_indices.extend(outlier_list_col)
    
  #print(outlier_indices)
  # 2(n) 개 이상의 이상치를 같는 index 선택
  outlier_indices = Counter(outlier_indices) # 요소별 개수를 새서 dict로 반환
  #print(type(outlier_indices))
  #print(outlier_indices)

  multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
  #print(multiple_outliers) # 이상치가 2(n)개 이상인 인덱스
  return multiple_outliers

# 제거해야할 인덱스를 얻는다.
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])
# 이상치 인덱스 제거
train = train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True) # index를 초기화

########## join train, test ######
train_len = len(train)
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

# np.nan으로 결측치 채우기
dataset = dataset.fillna(np.nan)

########## Embarked 결측치 S로 채움
dataset["Embarked"] = dataset["Embarked"].fillna("S")
# Embarked을 onehot인코딩해 수치형으로 바꿈
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")

########## 누락 된 Age를 SibSP, Parch, Pclass가 일치하는 행의 중간값으로 채운다.
index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)
for i in index_NaN_age :
    age_med = dataset["Age"].median()
    # 각 조건마다 인덱스가 True, False로 나온다.
    # Age가 결측치인행의 (SibSp 가 같은 행, Parch가 같은 행, Pclass가 같은행)을 
    # 교집합으로 찾아서 Age의 평균을 낸다
    age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & 
                               (dataset['Parch'] == dataset.iloc[i]["Parch"]) & 
                               (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) :
        dataset['Age'].iloc[i] = age_pred
    else :
        dataset['Age'].iloc[i] = age_med

########## Name으로부터 Title을 얻는다
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)

# Convert to categorical values Title 
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 
                                             'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 
                                             'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , 
                                         "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)
# Drop Name variable
dataset.drop(labels = ["Name"], axis = 1, inplace = True)
# Title을 onehot인코딩해 수치형으로 바꿈
dataset = pd.get_dummies(dataset, columns = ["Title"])

############ Create a family size descriptor from SibSp and Parch
dataset["Fsize"] = dataset["SibSp"] + dataset["Parch"] + 1 # 1은 자신
# Create new feature of family size
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if  s == 2  else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)

############ Cabin
# Cabin의 가장 앞글자를따오고 결측치는 X 로 한다
dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin'] ])
# one_hot incoding
dataset = pd.get_dummies(dataset, columns = ["Cabin"],prefix="Cabin")

############# Ticket
Ticket = []
for i in list(dataset.Ticket):
    if not i.isdigit() : # 접두사가 있다면
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
    else:
        Ticket.append("X")
        
dataset["Ticket"] = Ticket
# onehot_encoding
dataset = pd.get_dummies(dataset, columns = ["Ticket"], prefix="T")

############# Pclass
dataset["Pclass"] = dataset["Pclass"].astype("category")
dataset = pd.get_dummies(dataset, columns = ["Pclass"],prefix="Pc")

############# Fare
# 누락 된 값이 하나 있으므로 예측에 중요한 영향을 미치지 않는 중앙값으로 채우기
dataset["Fare"] = dataset["Fare"].fillna(dataset["Fare"].median())
# log변환
dataset['Fare'] = dataset['Fare'].map(lambda i : np.log(i) if i > 0 else 0)

############## Sex
dataset["Sex"] = dataset["Sex"].map({"male": 0, "female":1})

# 쓸모없는 Feature 삭제
dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)

dataset.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Em_C,Em_Q,Em_S,Title_0,Title_1,Title_2,Title_3,Fsize,Single,SmallF,MedF,LargeF,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X,T_A,T_A4,T_A5,T_AQ3,T_AQ4,T_AS,T_C,T_CA,T_CASOTON,T_FC,T_FCC,T_Fa,T_LINE,T_LP,T_PC,T_PP,T_PPP,T_SC,T_SCA3,T_SCA4,T_SCAH,T_SCOW,T_SCPARIS,T_SCParis,T_SOC,T_SOP,T_SOPP,T_SOTONO2,T_SOTONOQ,T_SP,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_X,Pc_1,Pc_2,Pc_3
0,0.0,0,22.0,1,0,1.981001,0,0,1,0,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1.0,1,38.0,1,0,4.266662,1,0,0,0,1,0,0,2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1.0,1,26.0,0,0,2.070022,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,1.0,1,35.0,1,0,3.972177,0,0,1,0,1,0,0,2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
4,0.0,0,35.0,0,0,2.085672,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [ ]:
train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=['Survived'], axis=1, inplace = True)

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0
NFOLDS = 5
kf = KFold(n_splits = NFOLDS, random_state = SEED)

# classifier, parameter를 매개변수로 받는다.
# 생성자로 classifier를 정의한다.
class SklearnHelper(object):
  def __init__(self, clf, seed = 0, params = None):
    params['random_state'] = seed
    self.clf = clf(**params)

  def train(self, x_train, y_train):
    self.clf.fit(x_train, y_train)

  def predict(self, x):
    return self.clf.predict(x)

  def fit(self, x ,y):
    return self.clf.fit(x,y)
  
  def feature_importances(self, x, y):
    print(self.clf.fit(x,y).feature_importances_)


In [ ]:
def get_oof(clf, x_train, y_train, x_test):
  oof_train = np.zeros((ntrain,)) # (891,)
  oof_test = np.zeros((ntest,)) # (418,)
  oof_test_skf = np.empty((NFOLDS, ntest)) # (5, 418)


  for i, (train_index, test_index) in enumerate(kf.split(x_train)):
    # train을 kfold로 나눠서 x_tr은 학습, x_te는 validation용 test로 사용한다
    print('i= ', i)
    x_tr = x_train[train_index]
    y_tr = y_train[train_index]
    x_te = x_train[test_index] # kfold로 인해 나눠진 test_index, 원래 train이였다.

    clf.train(x_tr, y_tr)

    # validation용으로 나눈 x_te를 예측 후 oof_train의 해당하는 index에 넣는다.
    # kfold정의시 shuffle을 False로 주었기때문에 kfold마다 겹치지않는다.
    oof_train[test_index] = clf.predict(x_te)
    
    # i번째 행에 kfold로 나뉜 4개의 데이터셋으로 학습한 모델의 예측을 넣는다.
    # 예측을 위해 넣은 데이터는 같지만, 모델학습에 사용한 데이터가 다르다.
    oof_test_skf[i,:]  = clf.predict(x_test)

  oof_test[:] = oof_test_skf.mean(axis=0) # x축 방향으로 평균을 낸다(세로) (5,418) -> (1,418)
  # oof_train : train을 kfold로 나눠서 예측한 결과
  # oof_test : 실제 test를 예측한 결과
  return oof_train.reshape(-1,1), oof_test.reshape(-1,1) # (891,1) , (418,1)

In [ ]:
# random forest parameters
rf_params = {
    'n_jobs' : -1,
    'n_estimators' : 500,
    'warm_start' : True,
    'max_depth' : 6,
    'min_samples_leaf' : 2,
    'max_features' : 'sqrt',
    'verbose' : 0
}

# Extra trees parameters
et_params = {
    'n_jobs' : -1,
    'n_estimators' : 500,
    'max_depth' : 8,
    'min_samples_leaf' : 2,
    'verbose' : 0
}

#AdaBoost parameters
ada_params = {
    'n_estimators' : 500,
    'learning_rate' : 0.75,
}

# Gradient boost parameters
gb_params = {
    'n_estimators' : 500,
    'max_depth' : 5,
    'min_samples_leaf' : 2,
    'verbose' : 0
}

# Support vector Classifier parameters
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
}

In [ ]:
# create 5 objects 
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf = AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf = SVC, seed=SEED, params=svc_params)

In [ ]:
y_train = train['Survived'].ravel() # 평평하게 해주는 함수
train = train.drop(['Survived'], axis = 1)
x_train = train.values # 굳이 numpy로 추출해서 넣지 않아도 된다.
x_test = test.values

In [ ]:
RandomF = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
RandomF.fit(train,y_train)
RandomF.predict(test)

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [ ]:
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test)
rf_oof_train, rf_oof_test = get_oof(rf, x_train, y_train, x_test)
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test)
gb_oof_train, gb_oof_test = get_oof(gb, x_train, y_train, x_test)
svc_oof_train, svc_oof_test = get_oof(svc, x_train, y_train, x_test)

print('Traning is complete')

i=  0
i=  1
i=  2
i=  3
i=  4
i=  0
i=  1
i=  2
i=  3
i=  4
i=  0
i=  1
i=  2
i=  3
i=  4
i=  0
i=  1
i=  2
i=  3
i=  4
i=  0
i=  1
i=  2
i=  3
i=  4
Traning is complete


In [ ]:
et_oof_train.shape, et_oof_test.shape

((874, 1), (418, 1))

In [ ]:
et_oof_train[:10], et_oof_test[:10]

(array([[0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]]), array([[0. ],
        [0.6],
        [0. ],
        [0. ],
        [1. ],
        [0. ],
        [1. ],
        [0. ],
        [1. ],
        [0. ]]))

# Second-Level Predictions from the First-level Output

In [ ]:
x_train = np.concatenate(( 
    et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( 
    et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

In [ ]:
x_train.shape, x_test.shape

((874, 5), (418, 5))

In [ ]:
x_train[:10], x_test[:10]

(array([[0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]), array([[0. , 0. , 0. , 0.2, 0. ],
        [0.6, 1. , 0.6, 0. , 1. ],
        [0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0.8, 0. ],
        [1. , 1. , 0.4, 0.4, 1. ],
        [0. , 0. , 0. , 0. , 0. ],
        [1. , 1. , 1. , 0. , 1. ],
        [0. , 0. , 0. , 0. , 0. ],
        [1. , 1. , 1. , 1. , 1. ],
        [0. , 0. , 0. , 0. , 0. ]]))

In [ ]:
gbm = xgb.XGBClassifier(
    # learning_rate = 0.02,
    n_estimators = 2000,
    max_depth=4,
    min_child_weight = 2,
    # gamma=1,
    gamma=0.9,
    subsample = 0.8,
    colsample_bytree=0.8,
    objective = 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

In [ ]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
StackingSubmission.to_csv("StackingSubmission_use_Titanic_3_notebook.csv", index=False)

In [ ]:
StackingSubmission

,PassengerId,Survived
0,892,0.0
1,893,1.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [ ]:
pd.read_csv('/content/gender_submission.csv')

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
